# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [ ]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset admission predict from [here](https://drive.google.com/file/d/1y-8TOlbiSWUoMZ_Sg-WfOL2NySXP6i4e/view?usp=sharing]), place it in the data folder and load it

In [ ]:
# your code here
admissions = pd.read_csv('../data/Admission_Predict.csv')

Remove trailing spaces at the end of the column names if there are any.


In [ ]:
# your code here
admissions.rename(columns=lambda x: x.strip(), inplace=True)

Let's evaluate the dataset by looking at the `head` function.

In [ ]:
# your code here
admissions.head()

Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [ ]:
# your code here
admissions.isnull().sum()

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [ ]:
# your code here
admissions.set_index('Serial No.', inplace = True, drop = False) 
admissions.head()

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [ ]:
# your code here

print(admissions.groupby(['GRE Score','CGPA']).ngroups)
print(admissions['Serial No.'].size)

# the number of unique combinations of GRE Score and CGPA is the same as the number of unique values in Serial No.

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [ ]:
admissions.set_index(['GRE Score', 'CGPA'], inplace = True) 
admissions.head()

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [ ]:
# your code here
admissions.reset_index(inplace = True)

In [ ]:
admissions.head()

# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [ ]:
# your code here
subset = admissions.loc[(admissions['CGPA'] > 9) & (admissions['Research'] == 1)]
subset.shape

#101 rows have this condition

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [ ]:
# your code here
subset2 = admissions.loc[(admissions['CGPA'] > 9) & (admissions['SOP'] < 3.5)]
subset2['Chance of Admit'].mean()


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [ ]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    return (col - np.mean(col))/ np.std(col)

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [ ]:
# your code here
admissions['CGPA_std'] = standardize(admissions['CGPA'])
admissions['GRE_std'] = standardize(admissions['GRE Score'])
admissions['LOR_std'] = standardize(admissions['LOR'])
admissions.head()

We will generate the decision choice at random using the code below. Please run the cell.

In [ ]:
# Libraries
from random import choices

In [ ]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [ ]:
# your code here
admissions['deciding_column'] = admissions.lookup(admissions.index, decision_choice)
admissions.head()

Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [ ]:
# your code here
admissions['decision'] = np.where(admissions['deciding_column'] > 0.8, 1, 0)
admissions.head()

How many applicants will be accepted to the program using the decision column? Compute the result below.

In [ ]:
# your code here
admissions['decision'].value_counts()

# 88 applicants will be accepted

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
import re

admissions.rename(columns= lambda x: re.sub(' ', '_', x).lower().strip(), inplace = True)
admissions.columns


In [ ]:
admissions.head()

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
# your code here
admissions['gre_score'] = np.where(admissions.university_rating >= 4, admissions.gre_score + 10, admissions.gre_score)

admissions.head()

In [ ]:
bin_labels = ['Low', 'Moderate', 'High', 'Very High']
admissions['adjusted_gre'] = pd.cut(admissions['gre_score'], 4, labels=bin_labels)

admissions.head()

In [ ]:
#Paolo: very good work! Only one suggestion: Here you modify the original gre_score directly (adding 10 points when university_rating>=4)
# but in doing so you modify the raw gre_score data. In similar situations it would be probably more transparent to add an intermediate column where you
# do the manipulation, leaving the raw gre_score data untouched. This to make clear what are the original data and what are the modifications you apply to them.